In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Load processed data
ticker = "BTC-USD"
df = pd.read_csv(f"{ticker}_processed.csv", index_col="Date", parse_dates=True)

# Features and target
features = ["Open", "High", "Low", "Close", "Volume", "Lag1", "Lag7", "SMA7", "RSI14", "MACD", "Returns"]
X = df[features]
y = df["Target"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
model.fit(X_train_scaled, y_train)

# Predict
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print(f"MSE for {ticker}: {mse}")
print(f"MAPE for {ticker}: {mape:.2f}%")

# Check directional accuracy
actual_direction = (y_test.shift(-1) > y_test).astype(int)  # 1 if price increases, 0 if decreases
predicted_direction = (pd.Series(y_pred, index=y_test.index).shift(-1) > pd.Series(y_pred, index=y_test.index)).astype(int)
directional_accuracy = (actual_direction == predicted_direction).mean() * 100
print(f"Directional Accuracy for {ticker}: {directional_accuracy:.2f}%")


# Preview predictions vs actual
results = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
print("\nPrediction Sample:")
print(results.head())

MSE for BTC-USD: 687481371.4572836
MAPE for BTC-USD: 24.51%
Directional Accuracy for BTC-USD: 49.65%

Prediction Sample:
                  Actual     Predicted
Date                                  
2024-10-15  67612.718750  66712.689824
2024-10-16  67399.835938  67578.992497
2024-10-17  68418.789062  67448.687259
2024-10-18  68362.734375  68165.814316
2024-10-19  69001.703125  68080.916619
